In [38]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
import torch.optim as optim


 ## 2. Préparer les transformations (prétraitement et augmentation)
👉 Les images doivent être mises à la même taille et normalisées pour ResNet. 👉 Pour l’entraînement, on ajoute des augmentations (flip, rotation) pour aider le modèle à mieux généraliser.

In [40]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


## 3 Chargement de la dataset avec ces transformations + Creation des Datalaoders :

In [41]:
train_dataset = datasets.ImageFolder(root='data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='data/val', transform=val_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


## 4 preparation de modéle (ResNet18) :

In [42]:
model = models.resnet18(pretrained=True)   # Charger un modèle pré-entraîné
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,2)  
criterion = nn.CrossEntropyLoss()  # fonction de perte pour la classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # optimiseur

## Entrainement de modéle : 

In [44]:
n_epochs = 5  # nombre d'époques
best_acc = 0.0  # pour sauvegarder le meilleur modèle

for epoch in range(n_epochs):
    print(f"--- Epoch {epoch+1}/{n_epochs} ---")
    
    # === Phase d'entraînement ===
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Train Loss: {running_loss/len(train_loader):.4f}")
    
    # === Phase de validation ===
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Validation Accuracy: {acc:.2f}%")
    
    # === Sauvegarde du meilleur modèle ===
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "model_best.pth")
        print("💾 Modèle sauvegardé ! (meilleur jusqu'ici)")


--- Epoch 1/5 ---
Train Loss: 0.4219
Validation Accuracy: 66.93%
💾 Modèle sauvegardé ! (meilleur jusqu'ici)
--- Epoch 2/5 ---
Train Loss: 0.2500
Validation Accuracy: 90.85%
💾 Modèle sauvegardé ! (meilleur jusqu'ici)
--- Epoch 3/5 ---
Train Loss: 0.1920
Validation Accuracy: 93.41%
💾 Modèle sauvegardé ! (meilleur jusqu'ici)
--- Epoch 4/5 ---
Train Loss: 0.1886
Validation Accuracy: 90.85%
--- Epoch 5/5 ---
Train Loss: 0.1606
Validation Accuracy: 71.65%
